# Question 1

In [1]:
!gcloud --version

Google Cloud SDK 369.0.0
alpha 2022.01.14
beta 2022.01.14
bq 2.0.72
core 2022.01.14
gsutil 5.6


# Question 2


Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.dataset will be created
  + resource "google_bigquery_dataset" "dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "trips_data_all"
      + delete_contents_on_destroy = false
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "europe-west6"
      + project                    = "dtc-de-339016"
      + self_link                  = (known after apply)

      + access {
          + domain         = (known after apply)
          + group_by_email = (known after apply)
          + role           = (known after apply)
          + special_group  = (known after apply)
          + user_by_email  = (known after apply)

          + view {
              + dataset_id = (known after apply)
              + project_id = (known after apply)
              + table_id   = (known after apply)
            }
        }
    }

  # google_storage_bucket.data-lake-bucket will be created
  + resource "google_storage_bucket" "data-lake-bucket" {
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "EUROPE-WEST6"
      + name                        = "dtc_data_lake_dtc-de-339016"
      + project                     = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + uniform_bucket_level_access = true
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "Delete"
            }

          + condition {
              + age                   = 30
              + matches_storage_class = []
              + with_state            = (known after apply)
            }
        }

      + versioning {
          + enabled = true
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.

Do you want to perform these actions?
  Terraform will perform the actions described above.
  Only 'yes' will be accepted to approve.

  Enter a value: yes

google_bigquery_dataset.dataset: Creating...
google_storage_bucket.data-lake-bucket: Creating...
google_storage_bucket.data-lake-bucket: Creation complete after 1s [id=dtc_data_lake_dtc-de-339016]
google_bigquery_dataset.dataset: Creation complete after 1s [id=projects/dtc-de-339016/datasets/trips_data_all]

Apply complete! Resources: 2 added, 0 changed, 0 destroyed.


# Question 3

In [3]:
!docker-compose up -d

Starting week1_pgdatabase_1 ... 
Starting week1_pgadmin_1    ... 
ting week1_pgadmin_1    ... done

SELECT 
  count(tpep_pickup_datetime)
FROM 
  yellow_taxi_trips t
WHERE
  date_trunc('day', t."tpep_pickup_datetime") = to_date('2021-01-15','YYYY-MM-DD');

53024

# Question 4

SELECT 
  CAST(tpep_pickup_datetime AS DATE) AS "day",
  max(tip_amount)
FROM 
  yellow_taxi_trips t
GROUP BY
  CAST(tpep_pickup_datetime AS DATE)
ORDER BY max ASC;

1140.44 on 2021-01-20

# Question 5

SELECT
  "Zone",
  count(*)
FROM (SELECT 
  tpep_pickup_datetime,
  "PULocationID",
  COALESCE(zpu."Zone", 'Unknown') AS "pickup_loc",
  COALESCE(zdo."Zone", 'Unknown') AS "dropoff_loc",
  zdo."Zone" AS "Zone"
FROM
  yellow_taxi_trips t
JOIN
  zones zpu
ON
  t."PULocationID" = zpu."LocationID"
JOIN
  zones zdo
ON
  t."DOLocationID" = zdo."LocationID"
WHERE
  zpu."Zone" = 'Central Park' AND
  date_trunc('day', t."tpep_pickup_datetime") = to_date('2021-01-14','YYYY-MM-DD')
) as a
GROUP BY 
  "Zone"
ORDER BY "count" ASC;

Upper East Side South - 97

# Question 6

SELECT
  "pickup_loc",
  "dropoff_loc",
  AVG("total_amount")
FROM (SELECT 
  tpep_pickup_datetime,
  "PULocationID",
  COALESCE(zpu."Zone", 'Unknown') AS "pickup_loc",
  COALESCE(zdo."Zone", 'Unknown') AS "dropoff_loc",
  zdo."Zone" AS "Zone",
  total_amount
FROM
  yellow_taxi_trips t
JOIN
  zones zpu
ON
  t."PULocationID" = zpu."LocationID"
JOIN
  zones zdo
ON
  t."DOLocationID" = zdo."LocationID"
) as a
GROUP BY
  "pickup_loc",
  "dropoff_loc"
ORDER BY "avg" ASC;

Manhattan/Alphabet City - Unknown - 2292.4